### Approach followed

##### To extract data in desired format, I've used regular expressions and python functions to iterate through each file in a line by line manner. Here are the major steps:
##### a) Read all files one by one
##### b) Go through each line to select the starting line and ending line as we have a lot of unwanted text data above and below the data to be extracted. Regular expressions and python functions have been used extensively to extract useful information
##### c) Once the starting point and ending point are finalized, store the filtered data in a dataframe
##### d) Iterate through the dataframe values extracted above and extract key-value pairs of entity name(asset, liabilities, etc.) as key and value as the respective amount 
##### e) Store the output one by one as per the desired dictionary format and add append as a new row in output dataframe export to CSV format (same can be applied to existing dataframe but just to keep the output clean, have created a new dataframe for outcome.)

<img src = "Instructions.jpg"/>

### Importing libraries

In [399]:
import pandas as pd
import re 
import numpy as np 
import os
from tqdm import tqdm

### Defining local functions and dataframe

In [400]:
# Creating this dataframe to store that filtered lines for each story and other columns to verify 
# the position from and till where the data is filtered. Also, to help in troubleshooting whether the data is filtered or not.

columns = {'Filename', 'FilteredData','IsStrippedFromStart','IsStrippedFromEnd','Stripped till Line No', 'Stripped till Line Text','Stripped from Line No', 'Stripped from line Text'}
df = pd.DataFrame(columns=columns)

In [401]:
# Clean unwanted brackets and comma in numbers/amount
def is_punct_custom(filename, text):
    
    cleaned_items = []
    extra_punct = ["{", "}", "(", ")","," ]
    curly_braces = ["{","}"]
    
    for t in text:
        new_str = ""
        for item in t.split():
            if any(x in item for x in extra_punct):
                old_item = item
                # remove all the brackets, commas and check the remaining value
                item = item.replace('{','').replace('}','').replace('(','').replace(')','').replace(',','')
                if not re.match('^[0-9]*$', item.strip()):  #if after removing the brackets & commas, the value becomes a string
                    item = old_item
                else:  # if the value is a digit after removing the brackets and commas
                    if any(braces in old_item for braces in curly_braces):  # check if any curly braces exists
                        item = old_item  #leave the curly braces one as it is as per required output
                    else:
                        item = "-"+item  # else add hyphen to the digits
                        
            new_str += item + " "
        cleaned_items.append(new_str.strip())
    return cleaned_items

### Reading all text files one by one and filtering out the unwanted text

In [402]:
listoffiles = os.listdir("HCL ML Challenge Dataset/")

# iterate through all the files one by one and filter out the unnecessary content
for i, filename in tqdm(enumerate(listoffiles)):
    filename_complete = "HCL ML Challenge Dataset\\"+filename
    f = open(filename_complete, "r",encoding='utf-8')
    listoflines = f.readlines()
    start_point = -1
    start_point_value = ""
    end_point = -1
    end_point_value = -1
    filtered_lines = list()
    
    # Iterate through each line of the text and filter out the unnecessary content
    for index,l in enumerate(listoflines):
        if l.rstrip():   # To exclude blank lines 
            l = re.sub('\s+', ' ', l.strip())  # remove unwanted white spaces
            regex = re.compile('[=@_!#%^&*<>?/\|~]')  # remove the lines with special characters in text
            regex_currency_symbol = re.match('^[£€ ]*$', l)  # remove lines containing only currencies
            if regex.search(l) == None and regex_currency_symbol == None: 
                filtered_lines.append(l)
    
    # set the starting point to the next line from where there is a match
    for index, l in enumerate(filtered_lines):
        if "as at" in l.lower() or "balance sheet at" in l.lower():
            start_point = index
            start_point_value = l
            break
    
    # If no starting point exists, then set the starting point as the first line else set to the next line
    if start_point == -1:
        start_point = 0
    else:
        start_point = start_point + 1
        
    iterat = start_point    
    while iterat < len(filtered_lines):
        words = filtered_lines[iterat].split()
        word_count = 0
        numeric_count = 0
        for word in words:
            if not re.match('^[0-9]*$', word): 
                word_count += 1
            if word.isdigit():
                numeric_count += 1
        # exclude the line and lines below where there are more than string words                         
        if word_count > 10 :
            end_point = iterat
            end_point_value = filtered_lines[iterat]
            break
        iterat += 1
        
    is_stripped_from_start = True if start_point != -1 else False
    is_stripped_from_end = True if end_point != -1 else False
    
    # If no end_point has been filtered out earlier, thenset the end point to the last line of text
    if end_point == -1:
        end_point = len(listoflines)
        end_point_value = listoflines[-1]
    
    # Check if the word contains year like 2019 or 2018, etc. then set the starting point as the next line
    if is_stripped_from_start:
        match = re.match(r'.*([1-2][0-9]{3})', filtered_lines[start_point])  
        if match is None:  # Set the starting point to next line
            # Then it found a match!
            start_point = start_point+1
            start_point_value = filtered_lines[start_point+1]

    if is_stripped_from_end:
        # exclude the line and lines below with following set of keywords
        except_last_lines = ['statement', 'average', 'notes', 'number', 'information', 'activity', ':', 'employee']
        if any(exception in filtered_lines[end_point-1].lower() for exception in except_last_lines):
            end_point = end_point-1
            if any(exception in filtered_lines[end_point-1].lower() for exception in except_last_lines):
                end_point = end_point-1
        end_point_value = filtered_lines[end_point]
    
    clean_lines = filtered_lines[start_point: end_point]
    
    # remove unwated brackets, commas from the text lines
    final_lines = is_punct_custom(filename, clean_lines)
    
    df = df.append({'Filename': filename, 'FilteredData':final_lines, 'IsStrippedFromStart':is_stripped_from_start,
                    'IsStrippedFromEnd':is_stripped_from_end, 'Stripped till Line No': start_point, 
                    'Stripped till Line Text': start_point_value, 'Stripped from Line No': end_point, 
                    'Stripped from line Text': end_point_value},ignore_index=True)


0it [00:00, ?it/s]
7it [00:00, 62.23it/s]
15it [00:00, 66.16it/s]
22it [00:00, 66.43it/s]
29it [00:00, 66.87it/s]
35it [00:00, 62.02it/s]
41it [00:00, 61.06it/s]
48it [00:00, 63.20it/s]
54it [00:00, 60.33it/s]
61it [00:00, 62.07it/s]
69it [00:01, 65.25it/s]
77it [00:01, 67.70it/s]
84it [00:01, 67.33it/s]
93it [00:01, 70.66it/s]
101it [00:01, 68.01it/s]
108it [00:01, 65.98it/s]
117it [00:01, 69.21it/s]
124it [00:01, 66.48it/s]
132it [00:01, 68.09it/s]
140it [00:02, 71.02it/s]
148it [00:02, 71.69it/s]
156it [00:02, 69.61it/s]
164it [00:02, 71.34it/s]
172it [00:02, 72.13it/s]
180it [00:02, 72.84it/s]
188it [00:02, 72.28it/s]
196it [00:02, 66.85it/s]
204it [00:02, 68.75it/s]
211it [00:03, 67.97it/s]
218it [00:03, 68.15it/s]
225it [00:03, 67.80it/s]
233it [00:03, 69.17it/s]
241it [00:03, 70.01it/s]
249it [00:03, 69.36it/s]
256it [00:03, 66.87it/s]
263it [00:03, 67.07it/s]
270it [00:03, 65.52it/s]
278it [00:04, 67.71it/s]
285it [00:04, 67.71it/s]
293it [00:04, 68.94it/s]
301it [00:04, 69.69

### Extracting details in required format and creating output

In [406]:
# creating the dataframe to export the output as csv
df_final = pd.DataFrame(columns = {'Filename', 'Extracted Values'})

In [407]:
# iterate through each of the text file output from above dataframe and extarct info as per desired format  
for index, filtered_data in enumerate(df.FilteredData[0:len(df.FilteredData)]):
    result = ""
    exists = False
    total_lenth = 0
    exists_at = -1     
    
    # Remove the text from 1st line where year or "note(s)" doesn't exist
    new_first_row = ""
    for index_1, item in enumerate(filtered_data[0].split()):
        match = re.match(r'\bnote?|([1-2][0-9]{3})', item.lower())
        if match is not None:
            new_first_row += item + " "

    new_first_row = new_first_row.strip()
    total_length = len(new_first_row.split())
    
    # check if 2019 exists in the first line 
    for index_1,item in enumerate(new_first_row.split()):
        if item == '2019':
            exists = True  # To be used later on while extracting amount values
            exists_at = index_1
    
    # Generate the final filtered key-value pairs
    for value in filtered_data[1:]:
        output = ""
        digits = []
        line_length = len(value.split())
        
        # extract text and respective amount one by one
        for i, val in enumerate(value.split()): 
            # if is a collection of numbers, - or .
            match = re.match(r'.*([\d.-]+$)', val)
            
            if match:
                # if is the last item, then set as amount
                if i == line_length-1:
                    digits.append(val)
                    
                # if is not the last item and is a amount, check if the next item is text or numbers
                if i+1 <= line_length-1:                    
                    # if next one is also a combination of digits, consider it as the value else as string 
                    if re.match(r'.*([\d.-]+$)',value.split()[i+1]):
                        if val != '-' and val != ' ' and val != '':
                            val = val.split('.')[0]
                        digits.append(str(val))
                    else:
                        output +=  val +" "                        
            else:
                output +=  val +" "

        # if 2019 does not exist in the first line, set the amounts value as nan for all
        year_value = ""
        if exists == False or len(digits) == 0:
            year_value = "nan"
        else:
            if total_length == len(digits):
                year_value = digits[exists_at]
            elif total_length > len(digits):
                temp_length = total_length
                
                # prepend zero(es) if number of values are less than the first row, 
                # to make the indexes identical to the first row of 2019, 2018, Notes, etc. 
                while temp_length > len(digits):
                    digits.insert(0,'0')
                    temp_length -= 1
                year_value = digits[exists_at]
        
        # set the value as "nan" if there is no value
        if output.strip() == '':
                output = 'nan'
        
        # formatting as per the required outcome and replacing the currency symbols with correspoding HTML code 
        result += ('"'+output.strip() + '":"'+ year_value +'",').replace('£','&#163').replace('€','&#8364').replace('$','&#36')
    
    # preparing the dictionary formatting and adding to the dataframe
    df_final = df_final.append({'Filename':df.Filename[index],'Extracted Values':"{"+result.rstrip(',')+"}"},ignore_index=True)


### Formatting and exporting the results to CSV format

In [415]:
# removing '.txt' from filename as per the requirement
df_final = df_final[['Filename','Extracted Values']]
df_final.Filename = [filename.split('.')[0] for filename in df_final.Filename]
df_final.to_csv('output_updated.csv', header = True,index =False)

### Verifying that the values are being read as dictionary 

In [413]:
import json

In [414]:
for i in range(len(df_final["Extracted Values"])):
    print(json.loads(df_final["Extracted Values"][i][0:]))

{'Current Assets': '312', 'Creditors: amounts falling due within one year': '-59', 'Net current assets (liabilities)': '253', 'Total assets less current liabilities': '253', 'Total net assets (liabilities)': '253', 'Capital and reserves': '253'}
{'Current assets': 'nan', 'Cash at bank and in hand': '1000', 'Net assets': '1000', 'Issued share capital': 'nan', '1000 Ordinary Shares 1 of &#163 each': '1000', 'Total Shareholder funds': '1000'}
{'Called up share capital not paid': '-', 'Fixed Assets': '516', 'Current Assets': '-', 'Prepayments and accrued income': '-', 'Creditors: amounts falling due within one year': '0', 'Net current assets (liabilities)': '0', 'Total assets less current liabilities': '516', 'Creditors: amounts falling due after more than one year': '0', 'Provisions for liabilities': '0', 'Accruals and deferred income': '0', 'Total net assets (liabilities)': '516', 'Capital and reserves': '516'}
{'Current Assets': '-23349', 'Creditors: amounts falling due within one year'

{'Called up share capital not paid': '1', 'Current Assets': '-1016', 'Net current assets (liabilities)': '-1016', 'Total assets less current liabilities': '-1017', 'Total net assets (liabilities)': '-1017', 'Capital and reserves': '-1017'}
{'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Issued share capital': 'nan', 'Total Shareholder funds': '100'}
{'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Issued share capital': 'nan', '100 Ordinary Shares 1 of &#163 each': '100', 'Total Shareholder funds': '100'}
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', '1 Ordinary Share 1 of &#163 each': '1', 'Total Shareholder funds': '1'}
{'Called up share capital not paid': '-', 'Fixed Assets': '-', 'Current Assets': '-', 'Net current assets (liabilities)': '0', 'Total assets less current liabilities': '0', 'Total net assets (liabilities)': '0', 'Capital and reserves': '0

{'Called up share capital not paid': '-', 'Fixed Assets': '-', 'Current Assets': '-', 'Prepayments and accrued income': '-', 'Creditors: amounts falling due within one year': '0', 'Net current assets (liabilities)': '0', 'Total assets less current liabilities': '0', 'Creditors: amounts falling due after more than one year': '0', 'Provisions for liabilities': '0', 'Accruals and deferred income': '0', 'Total net assets (liabilities)': '0', 'Capital and reserves': '0'}
{'Current Assets': '-22560', 'Creditors: amounts falling due within one year': '-2038', 'Net current assets (liabilities)': '-20522', 'Total assets less current liabilities': '-20522', 'Total net assets (liabilities)': '-20522', 'Capital and reserves': '-20522'}
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', '1 Ordinary Share 1 of &#163 each': '1', 'Total Shareholder funds': '1'}
{'Fixed Assets': '-46754', 'Current Assets': '-8017', 'Creditors: amounts falling du

{'Fixed Assets': 'nan', 'Current Assets': 'nan', 'Creditors: amounts falling due within one year': 'nan', 'Net current assets (liabilities)': 'nan', 'Total assets less current liabilities': 'nan', 'Creditors: amounts falling due after more than one': 'nan', 'nan': 'nan', 'year': 'nan', 'Total net assets (liabilities)': 'nan', 'Capital and reserves': 'nan'}
{'Called up share capital not paid': '1', 'Net assets': '1', 'Tssued share capital': 'nan', '1 Ordinary Share 1 of &#163 each': '1', 'Total Shareholder funds': '1'}
{'Called up share capital not paid': '500', 'Net assets': '500', 'Tssued share capital': 'nan', '500 Ordinary Shares 1 of &#163 each': '500', 'Total Shareholder funds': '500'}
{'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Issued share capital': 'nan', '100 Ordinary Shares 1 of &#163 each': '100', 'Total Shareholder funds': '100'}
{'Net assets': '0', 'Reserves': '0'}
{'Net assets': '0', 'Reserves': '0'}
{'Called up share capital not pai

{'Called up share capital not paid': '1', 'Fixed Assets': '-', 'Current Assets': '-10000', 'Prepayments and accrued income': '-', 'Creditors: amounts falling due within one year': '0', 'Net current assets (liabilities)': '-10000', 'Total assets less current liabilities': '-10001', 'Creditors: amounts falling due after more than one year': '0', 'Provisions for liabilities': '0', 'Accruals and deferred income': '0', 'Total net assets (liabilities)': '-10001', 'Capital and reserves': '-10001'}
{'Fixed Assets': '-2046', 'Current Assets': '-53790', 'Creditors: amounts falling due within one year': '-23146', 'Net current assets (liabilities)': '-30644', 'Total assets less current liabilities': '-32690', 'Total net assets (liabilities)': '-32690', 'Capital and reserves': '-32690'}
{'Fixed Assets': '-14800', 'Current Assets': '570', 'Net current assets (liabilities)': '570', 'Total assets less current liabilities': '-15370', 'Creditors: amounts falling due after more than one year': '-18659', 

...

{'Called up share capital not paid': '100', 'Net assets': '100', 'Tssued share capital': 'nan', '100 Ordinary Shares 1 of &#163 each': '100', 'Total Shareholder funds': '100'}
{'Called up share capital not paid': '100', 'Net assets': '100', 'Tssued share capital': 'nan', 'Total Shareholder funds': '100'}
{'Current Assets': '-2124', 'Creditors: amounts falling due within one year': '-250', 'Net current assets (liabilities)': '-1874', 'Total assets less current liabilities': '-1874', 'Creditors: amounts falling due after more than one': 'nan', '(90.628) (90.628)': 'nan', 'year': 'nan', 'Total net assets (liabilities)': '-88754', 'Capital and reserves': '-88754'}
{'Called up share capital not paid': '1', 'Current Assets': '320', 'Prepayments and accrued income': '135', 'Net current assets (liabilities)': '455', 'Total assets less current liabilities': '456', 'Total net assets (liabilities)': '456', 'Capital and reserves': '456'}
{'Fixed Assets': '53', 'Current Assets': '-3405', 'Creditors

{'Fixed Assets': '-1090', 'Current Assets': '-9037', 'Creditors: amounts falling due within one year': '-8000', 'Net current assets (liabilities)': '-1037', 'Total assets less current liabilities': '-2127', 'Accruals and deferred income': '-1100', 'Total net assets (liabilities)': '-1027', 'Capital and reserves': '-1027'}
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', 'Total Shareholder funds': '1'}
{'Called up share capital not paid': '0', 'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Authorised share capital': 'nan', '0 Ordinary of &#1631 each': 'nan', 'Issued share capital': 'nan', '100 Ordinary of &#1631 each': '100', 'Total shareholder funds': '100'}
{'Called up share capital not paid': '100', 'Fixed Assets': '-', 'Current Assets': '-5659', 'Prepayments and accrued income': '-', 'Creditors: amounts falling due within one year': '-4659', 'Net current assets (liabilities)': '-1000', 'To

{'Called up share capital not paid': '1', 'Net assets': '1', 'Tssued share capital': 'nan', '1 Ordinary Share 1 of &#163 each': '1', 'Total Shareholder funds': '1'}
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', '1 Ordinary Share 1 of &#163 each': '1', 'Total Shareholder funds': '1'}
{'Fixed Assets': '522', 'Current Assets': '-17165', 'Creditors: amounts falling due within one year': '-17234', 'Net current assets (liabilities)': '-69', 'Total assets less current liabilities': '453', 'Provisions for liabilities': '-99', 'Total net assets (liabilities)': '354', 'Capital and reserves': '354'}
{'Current assets': 'nan', 'Cash at bank and in hand': '5', 'Net assets': '5', 'Issued share capital': 'nan', '5 Ordinary Shares 1 of &#163 each': '5', 'Total Shareholder funds': '5'}
{'Called up share capital not paid': '100', 'Net assets': '100', 'Tssued share capital': 'nan', '100 Ordinary Shares 1 of &#163 each': '100', 'Total Sharehold

{'Called up share capital not paid': '0', 'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Authorised share capital': 'nan', '100 ORDINARY SHARES of &#1631 each': '100', 'Issued share capital': 'nan', 'Total shareholder funds': '100'}
{'Current Assets': '-1132', 'Net current assets (liabilities)': '-1132', 'Total assets less current liabilities': '-1132', 'Creditors: amounts falling due after more than one year': '-5000', 'Total net assets (liabilities)': '-3868', 'Capital and reserves': '-3868'}
{'Fixed Assets': '-1391', 'Current Assets': '917', 'Creditors: amounts falling due within one year': '-2164', 'Net current assets (liabilities)': '-1247', 'Total assets less current liabilities': '144', 'Total net assets (liabilities)': '144', 'Capital and reserves': '144'}
{'Called up share capital not paid': '1', 'Net assets': '1', 'Tssued share capital': 'nan', '1 Ordinary Share 1 of &#163 each': '1', 'Total Shareholder funds': '1'}
{'Fixed Assets': '-2342',

{'Current Assets': '-4387', 'Creditors: amounts falling due within one year': '-350', 'Net current assets (liabilities)': '-4037', 'Total assets less current liabilities': '-4037', 'Creditors: amounts falling due after more than one year': '-35600', 'Total net assets (liabilities)': '-31563', 'Capital and reserves': '-31563'}
{'Net assets': '0', 'Reserves': '0'}
{'Called up share capital not paid': '2', 'Net assets': '2', 'Tssued share capital': 'nan', '2 Ordinary Shares 1 of &#163 each': '2', 'Total Shareholder funds': '2'}
{'Fixed Assets': '-', 'Current Assets': '100', 'Creditors: amounts falling due within one year': '0', 'Net current assets (liabilities)': '100', 'Total assets less current liabilities': '100', 'Accruals and deferred income': '0', 'Total net assets (liabilities)': '100', 'Capital and reserves': '100'}
{'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Issued share capital': 'nan', '100 Ordinary Shares 1 of &#163 each': '100', 'Total S